<a href="https://colab.research.google.com/github/Jessicantma/TaskWeek6/blob/main/Twitter_Sentiment_Analysis_by_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jp797498e_twitter_entity_sentiment_analysis_path = kagglehub.dataset_download('jp797498e/twitter-entity-sentiment-analysis')

print('Data source import complete.')


100%|██████████| 1.99M/1.99M [00:00<00:00, 2.57MB/s]

Extracting files...


Data source import complete.


In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df_trian=pd.read_csv('/content/twitter_training.csv (1).zip')
df_val=pd.read_csv('/content/twitter_validation (1).csv')

In [4]:
df_trian.columns #error in colunms name

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

# cleaning data


In [5]:
# Remove duplicate rows
df_trian = df_trian.drop_duplicates()
df_val = df_val.drop_duplicates()

# Remove rows with empty values
df_trian = df_trian.dropna()
df_val = df_val.dropna()

# Rename columns
df_trian.columns = ["header","company","label","text"]
df_val.columns = ["header","company","label","text"]

In [6]:
df_trian

,header,company,label,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [7]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'http\S+', '', text)  # Remove URLs
  text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
  text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Remove hashtags
  text = re.sub(r'\W', ' ', text)  # Remove special characters
  text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
  text = text.strip()

  tokens = word_tokenize(text)
  filtered_tokens = [w for w in tokens if not w in stop_words]

  return " ".join(filtered_tokens)

df_trian['text'] = df_trian['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)

In [8]:
# Drop 'header' and 'company' columns
df_trian = df_trian.drop(['header', 'company'], axis=1)
df_val = df_val.drop(['header', 'company'], axis=1)

In [9]:
# Split df_val into df_test and df_val
df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)  # Adjust test_size as needed

print("df_val shape:", df_val.shape)
print("df_test shape:", df_test.shape)

df_val shape: (499, 2)
df_test shape: (500, 2)


In [10]:
le = LabelEncoder()

# Fit the encoder on the training data and transform the label column
df_trian['label'] = le.fit_transform(df_trian['label'])

# Transform the label column in df_val using the same encoder
df_val['label'] = le.transform(df_val['label'])

# Transform the label column in df_test using the same encoder
df_test['label'] = le.transform(df_test['label'])

In [11]:
# Tokenize the text data
max_words = 20000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df_trian['text'])

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(df_trian['text'])
val_sequences = tokenizer.texts_to_sequences(df_val['text'])
test_sequences = tokenizer.texts_to_sequences(df_test['text'])

# Pad sequences to a fixed length
max_sequence_length = 100
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')


In [12]:
embedding_dim = 128
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_sequence_length),
    Bidirectional(LSTM(64, return_sequences=True)),  # Return sequences for the next LSTM layer
    Bidirectional(LSTM(32)),  # Final LSTM layer
    Dense(64, activation='relu'),  # Add a dense layer for further processing
    Dense(4, activation='softmax')  # Output layer with softmax for multi-class classification
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
epochs = 10
batch_size = 32
model.fit(train_padded, df_trian['label'], epochs=epochs, batch_size=batch_size, validation_data=(val_padded, df_val['label']))

Epoch 1/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 492s 216ms/step - accuracy: 0.5920 - loss: 0.9684 - val_accuracy: 0.9299 - val_loss: 0.2309
Epoch 2/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 499s 215ms/step - accuracy: 0.8878 - loss: 0.3063 - val_accuracy: 0.9419 - val_loss: 0.1608
Epoch 3/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 502s 214ms/step - accuracy: 0.9198 - loss: 0.2047 - val_accuracy: 0.9519 - val_loss: 0.1450
Epoch 4/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 479s 214ms/step - accuracy: 0.9332 - loss: 0.1679 - val_accuracy: 0.9439 - val_loss: 0.1611
Epoch 5/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 500s 213ms/step - accuracy: 0.9447 - loss: 0.1377 - val_accuracy: 0.9599 - val_loss: 0.1182
Epoch 6/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 502s 213ms/step - accuracy: 0.9495 - loss: 0.1201 - val_accuracy: 0.9559 - val_loss: 0.1540
Epoch 7/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 505s 214ms/step - accuracy: 0.9545 - loss: 0.1065 - val_accuracy: 0.9619 - val_loss: 0.1620
Epoch 8/10
2240/2240 ━━━━━━━━━━━━━━━━━━━━ 500s 214ms/step - ac

In [15]:
loss, accuracy = model.evaluate(test_padded, df_test['label'])
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9762 - loss: 0.1307
Test Loss: 0.1768166422843933
Test Accuracy: 0.9679999947547913
